In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from text_untils import *
import pickle
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open('output/bm25/texts.pkl', 'rb') as f:
    texts = pickle.load(f)

with open('output/bm25/bm25plus.pkl', 'rb') as f:
    bm25plus = pickle.load(f)

In [3]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/docs.jsonl')

In [4]:
query = 'Tổ chức nào thực hiện trách nhiệm thông báo phòng chống khi xảy ra thiên tai'
# original_question = 'Cơ quan có trách nhiệm truyền tin phòng, chống thiên tai'
query = clean_text(query)
query = word_segment(query)
query = remove_stopword(normalize_text(query))

query_bm25 = query.split()
print(query_bm25)

['tổ_chức', 'thực_hiện', 'trách_nhiệm', 'thông_báo', 'phòng_chống', 'xảy', 'thiên_tai']


In [5]:
top_docs = bm25plus.get_top_n(query_bm25, docs, n=100)
bm25_scores = bm25plus.get_scores(query_bm25)
doc_scores = np.sort(bm25_scores)[::-1][:100]

In [6]:
for doc_dict, score in zip(top_docs, doc_scores):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Score: {score}")

ID: 653a89021e06875841d152d9, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai, Score: 44.75947558940982
ID: 653a89191e06875841d1533b, Document: trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định thế_nào, Score: 42.88955544019096
ID: 653a893d1e06875841d15435, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định, Score: 42.88955544019096
ID: 653a8b391e06875841d15be5, Document: quy_định trách_nhiệm thông_báo cháy rừng xảy, Score: 42.3373308462243
ID: 653a9ebe1e06875841d1c26d, Document: cơ_quan trách_nhiệm truyền phòng_chống thiên_tai, Score: 39.863646134737465
ID: 653a86321e06875841d13683, Document: ban chỉ_đạo quốc_gia phòng_chống thiên_tai trách_nhiệm thực_hiện ứng_phó sự_cố thiên_tai tìm_kiếm cứu nạn, Score: 39.185296661032226
ID: 653a9ef51e06875841d1c4ad, Document: phòng_chống thiên_tai, Score: 39.12280438901149
ID: 653a892e1e06875841d153e9, Document: trách_nhiệm tài_chính đối_với quỹ ph

In [7]:
target_min = 0
target_max = 10

rescaled_scores = [score * (target_max / max(doc_scores)) for score in doc_scores]

for i, (doc_dict, score) in enumerate(zip(top_docs, rescaled_scores)):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Rescaled Score: {score:.4f}")


ID: 653a89021e06875841d152d9, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai, Rescaled Score: 10.0000
ID: 653a89191e06875841d1533b, Document: trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định thế_nào, Rescaled Score: 9.5822
ID: 653a893d1e06875841d15435, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định, Rescaled Score: 9.5822
ID: 653a8b391e06875841d15be5, Document: quy_định trách_nhiệm thông_báo cháy rừng xảy, Rescaled Score: 9.4589
ID: 653a9ebe1e06875841d1c26d, Document: cơ_quan trách_nhiệm truyền phòng_chống thiên_tai, Rescaled Score: 8.9062
ID: 653a86321e06875841d13683, Document: ban chỉ_đạo quốc_gia phòng_chống thiên_tai trách_nhiệm thực_hiện ứng_phó sự_cố thiên_tai tìm_kiếm cứu nạn, Rescaled Score: 8.7546
ID: 653a9ef51e06875841d1c4ad, Document: phòng_chống thiên_tai, Rescaled Score: 8.7407
ID: 653a892e1e06875841d153e9, Document: trách_nhiệm tài_chính đối_với quỹ phòng_chống thiê

In [8]:
# doc_scores = rescaled_scores

In [9]:
query_sbert = query.replace('_', ' ')
print(query_sbert)

tổ chức thực hiện trách nhiệm thông báo phòng chống xảy thiên tai


In [10]:
df = pd.read_json('data/cleaned/temp.jsonl', lines=True)

In [11]:
doc_ids = [doc_dict['id'] for doc_dict in top_docs]
selected_rows = df[df['_id'].isin(doc_ids)]

In [12]:
sentences_title = []
ids_title = []

for idx in selected_rows.index:
    sentences_title.append(selected_rows.at[idx, 'title'])
    ids_title.append(selected_rows.at[idx, '_id'])

In [13]:
model_title = SentenceTransformer('keepitreal/vietnamese-sbert')
embeddings_title = model_title.encode(sentences_title)

In [14]:
query_embedding_title = model_title.encode([query_sbert], convert_to_tensor=True)
query_embedding_title = query_embedding_title.cpu()

In [15]:
similarities_title = cosine_similarity(query_embedding_title, embeddings_title)

top_50_indices_title = np.argsort(similarities_title[0])[::-1][:50]

for idx in top_50_indices_title:
    similarity_score = similarities_title[0][idx]
    similar_sentence = sentences_title[idx]
    doc_id = ids_title[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 653a89231e06875841d15391, Similarity Score: 0.7428, Sentence: tổ chức nhiệm vụ ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn huyện
ID: 653a86321e06875841d13683, Similarity Score: 0.7327, Sentence: ban chỉ đạo quốc gia phòng chống thiên tai trách nhiệm thực hiện ứng phó sự cố thiên tai tìm kiếm cứu nạn
ID: 653a89231e06875841d15393, Similarity Score: 0.7191, Sentence: tổ chức nhiệm vụ ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn tỉnh
ID: 653a91681e06875841d17ad3, Similarity Score: 0.7129, Sentence: trách nhiệm ủy ban nhân dân ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn quy định
ID: 653a9ebe1e06875841d1c26d, Similarity Score: 0.7106, Sentence: cơ quan trách nhiệm truyền phòng chống thiên tai
ID: 653a89231e06875841d1538f, Similarity Score: 0.7079, Sentence: tổ chức nhiệm vụ ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn xã
ID: 653a9a251e06875841d1ab0d, Similarity Score: 0.7016, Sentence: tổ chức nhiệm vụ ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn tỉnh huyệ

In [16]:
sentences_anwser = []
ids_anwser = []

for idx in selected_rows.index:
    sentences_anwser.append(selected_rows.at[idx, 'answer'])
    ids_anwser.append(selected_rows.at[idx, '_id'])

In [17]:
model_anwser = SentenceTransformer('keepitreal/vietnamese-sbert')
embeddings_anwser = model_anwser.encode(sentences_anwser)

In [18]:
query_embedding_anwser = model_anwser.encode([query_sbert], convert_to_tensor=True)
query_embedding_anwser = query_embedding_anwser.cpu()

In [19]:
similarities_anwser = cosine_similarity(query_embedding_anwser, embeddings_anwser)

top_50_indices_anwser = np.argsort(similarities_anwser[0])[::-1][:50]

for idx in top_50_indices_anwser:
    similarity_score = similarities_anwser[0][idx]
    similar_sentence = sentences_anwser[idx]
    doc_id = ids_anwser[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 653a86a41e06875841d13cff, Similarity Score: 0.5955, Sentence: quỹ phòng chống thiên tai chi nội dung hỗ trợ hoạt động dự án khắc phục hậu quả phòng chống thiên tai hoạt động cảnh báo tuyên truyền thiên tai
ID: 653a9ebe1e06875841d1c26d, Similarity Score: 0.5901, Sentence: cơ quan trách nhiệm truyền phòng chống thiên tai quy định 3 nghị định 66 2014 nđ cp hướng dẫn luật phòng chống thiên tai đài truyền hình việt nam đài tiếng nói việt nam trách nhiệm phát văn bản chỉ đạo ứng phó thiên tai thủ tướng chính phủ ban chỉ đạo trung ương phòng chống thiên tai ủy ban quốcgia tìm kiếm cứu nạn sóng truyền hình phát thanh ủy ban nhân dân trách nhiệm chỉ đạo tổ chức phổ biến kịp thời chính xác văn bản chỉ đạo chỉ huy ứng phó thiên tai thủ tướng chính phủ ban chỉ đạo trung ương phòng chống thiên tai ủy ban quốc gia tìm kiếm cứu nạn ban chỉ huy phòng chống thiên tai tìm kiếm cứu nạn địa bàn cơ quan ngang cơ quan chính phủ trách nhiệm chỉ đạo hệ thống thông tin chuyên ngành phát thông tin liên quan

In [20]:
df_title = pd.DataFrame({
    'doc_id': [ids_title[idx] for idx in top_50_indices_title],
    'similarity_score_title': [similarities_title[0][idx] for idx in top_50_indices_title],
    'sentence_title': [sentences_title[idx] for idx in top_50_indices_title]
})

df_anwser = pd.DataFrame({
    'doc_id': [ids_anwser[idx] for idx in top_50_indices_anwser],
    'similarity_score_anwser': [similarities_anwser[0][idx] for idx in top_50_indices_anwser],
})

df_merged = pd.merge(df_title, df_anwser, on='doc_id')
df_merged['sbert_score'] = df_merged['similarity_score_title'] * df_merged['similarity_score_anwser']

df_merged_sorted = df_merged.sort_values(by='sbert_score', ascending=False)
top_20_rows = df_merged_sorted.head(20)

In [21]:
print(top_20_rows)

                      doc_id  similarity_score_title  \
4   653a9ebe1e06875841d1c26d                0.710572   
3   653a91681e06875841d17ad3                0.712936   
1   653a86321e06875841d13683                0.732691   
10  653a89241e06875841d153a3                0.680922   
9   653a89021e06875841d152d9                0.686280   
15  653a89231e06875841d1539d                0.617235   
16  653a8b4a1e06875841d15c47                0.617159   
12  653a8ce91e06875841d16377                0.670515   
6   653a9a251e06875841d1ab0d                0.701596   
23  653a9ef51e06875841d1c4ad                0.556785   
8   653a9ebf1e06875841d1c283                0.687837   
0   653a89231e06875841d15391                0.742775   
25  653a893d1e06875841d15433                0.556394   
21  653a893d1e06875841d15435                0.575992   
19  653a89191e06875841d1533b                0.578379   
32  653a86a41e06875841d13cff                0.502798   
13  653a8ce91e06875841d16375                0.65

In [22]:
df_bm25 = pd.DataFrame({
    'doc_id': [doc_dict['id'] for doc_dict in top_docs],
    'bm25_score': doc_scores
})

df_bm25_filtered = df_bm25[df_bm25['doc_id'].isin(top_20_rows['doc_id'])]
df_final = pd.merge(df_bm25_filtered, top_20_rows, on='doc_id')
df_final['final_score'] = df_final['sbert_score'] * df_final['bm25_score']
df_final_sorted = df_final.sort_values(by='final_score', ascending=False)
print(df_final_sorted.head(5))

                      doc_id  bm25_score  similarity_score_title  \
3   653a9ebe1e06875841d1c26d   39.863646                0.710572   
0   653a89021e06875841d152d9   44.759476                0.686280   
15  653a91681e06875841d17ad3   36.218505                0.712936   
4   653a86321e06875841d13683   39.185297                0.732691   
6   653a8b4a1e06875841d15c47   39.062184                0.617159   

                                       sentence_title  \
3    cơ quan trách nhiệm truyền phòng chống thiên tai   
0   quy định trách nhiệm tổ chức thực hiện kế hoạc...   
15  trách nhiệm ủy ban nhân dân ban chỉ huy phòng ...   
4   ban chỉ đạo quốc gia phòng chống thiên tai trá...   
6   phòng chống thiên tai ban chỉ đạo trung ương t...   

    similarity_score_anwser  sbert_score  final_score  
3                  0.590104     0.419312    16.715287  
0                  0.516808     0.354675    15.875076  
15                 0.588054     0.419245    15.184434  
4                  0.512